In [16]:
! pip install wheel
! pip install hazm
! pip install transformers
! pip install sentencepiece
! pip install numpy>=1.17
!pip install datasets
!pip install googletrans

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.5 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
      Successfully uninstalled fsspec-2024.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requ

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 4.6 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.0.0-py3-none-any.whl size=15718 sha256=34a2f72ce94a339d1aec28de5fb18f0891be109927a77542adf4fad413bd4fde
  Stored in directory: /root/.cache/pip/wheels/b3/81/ea/8b030407f8ebfc2f857814e086bb22ca2d4fea1a7be63652ab
Successfully built googletrans
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling chardet-5.2.0:
   

In [1]:
import glob
import os

import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, ConfusionMatrixDisplay

import datasets
from tqdm import tqdm
import time

import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer,  MT5ForConditionalGeneration, MT5Tokenizer

import matplotlib.pyplot as plt

tqdm.pandas()

In [2]:
parsinlu_ent = datasets.load_dataset("persiannlp/parsinlu_entailment")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/755 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1675 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/270 [00:00<?, ? examples/s]

In [5]:
test_df = parsinlu_ent['test'].to_pandas()
test_df.shape

(1675, 4)

In [4]:
labels = test_df['label'].unique()
print(labels)
labels = labels[:3]
print(labels)

['c' 'n' 'e' '-']
['c' 'n' 'e']


In [9]:
test_df.head()

,sent1,sent2,category,label
0,او را نیز بکشتند و پس از او، هیچ‌کس را نیافتند...,او را نیز بکشتند و پس از او پسرش را شایسته تری...,natural-wiki,c
1,آنها به عنوان (الف) رسید جبران خسارت طبقه بندی...,چیز خوبی است که آنها در این دسته قرار می گیرند.,translation-train,n
2,نعمت‌زاده همچنین با اشاره به ارتقاء کیفیت خودر...,به گفته‌ی وی تولید خودرو در مقایسه با سال‌های ...,natural-miras,n
3,صابر دین‌پژوه‌ رتبه اول علوم ریاضی و فنی کنکور...,رتبه‌ی اول علوم ریاضی و فنی کنکور از نتیجه‌ی ک...,natural-miras,c
4,مورد دیگری که باید ببینید ، فیلم خوابیدن با دش...,باید خوابیدن با دشمن را ببینی.,translation-train,e


In [10]:
!pip install -U "transformers==4.40.0" --upgrade
!pip install accelerate bitsandbytes

In [12]:
import transformers
import torch

model_id = "unsloth/llama-3-8b-Instruct-bnb-4bit"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={
        "torch_dtype": torch.float16,
        "quantization_config": {"load_in_4bit": True},
        "low_cpu_mem_usage": True,
    },
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/usr/local/lib/python3.10/dist-packages/transformers/quantizers/auto.py:159: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [13]:
def get_inference(prompt):
  messages = [
    {"role": "system", "content": ".تو یک دستیار ویرایشگر متن های فارسی هستی"},
    {"role": "user", "content": """{prompt}""".format(prompt = prompt)},
  ]

  prompt = pipeline.tokenizer.apply_chat_template(
          messages,
          tokenize=False,
          add_generation_prompt=True
  )

  terminators = [
      pipeline.tokenizer.eos_token_id,
      pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  outputs = pipeline(
      prompt,
      max_new_tokens=2000,
      eos_token_id=terminators,
      do_sample=True,
      temperature=0.9,
      top_p=0.8,
  )

  return(outputs[0]["generated_text"][len(prompt):])

In [11]:

PROMPT_FEW = """
هدف وظیفه‌ی استنتاج زبان طبیعی تشخیص رابطه‌ی نتیجه‌گیری بین یک <فرضیه> با توجه به یک <پیش‌فرض> است.
رابطه‌ یا برچسب میان آن‌ها می‌تواند یکی از سه نوع تناظر، تناقض یا ناشناخته باشد.
- تناظر : اگر جمله <فرضیه> به طور منطقی نتیجه‌ای از جمله <پیش‌فرض> باشد
- تناقض : اگر جمله <فرضیه> با جمله <پیش‌فرض> در تناقض باشد
- ناشناخته : اگر رابطه‌ای قطعی بین جمله <پیش‌فرض> و جمله <فرضیه> وجود نداشته باشد و هیچ تناظر یا تناقضی نتوان برقرار کرد

مثال:
<پیش‌فرض><sep><فرضیه>
<برچسب>:
    تناقض یا تناظر یا ناشناخته

<گرچه فیلم موفق بود اما نقش او در این میان نادیده گرفته شد.><sep><در سال ۱۹۴۹ در فیلم اسکاری نامه‌ای به سه همسر ظاهر شد.>
<برچسب>:
    ناشناخته

<من همیشه شنیده ام که شما انقلابیون زندگی را ارزان پنداشته اید ، اما به نظر می رسد وقتی پای زندگی خود شما در میان باشد، قضیه فرق میکند><sep><من دائماً شنیده ام که شما انقلابیون برای زندگی بسیار ارزش قائل هستید.>
<برچسب>:
    تناقض

<در پایان او آهی طولانی سر داد.><sep><او در پایان آهسته آهی کشید.>
<برچسب>:
    تناظر

<در پس این حمله، کیافخرالدین جلال و سپس کیاوشتاسف به همراه فرزندانشان کشته‌شدند.><sep><آنها ازین مبارزه جان سالم بدر می برند.>
<برچسب>:
    تناقض

<باغ وحش ادینبورگ هر روز در طول تابستان رژه پنگوئن ها برگزار می کند.><sep><همه بازدید کنندگان تابستانی باغ وحش ادینبورگ، رژه پنگوئن ها را می بینند.>
<برچسب>:
    ناشناخته

<اخیرا برخی از نویسندگان اطلاعات مختصری از این قلعه ارائه داده‌اند که بیشتر منحصر به شکل ظاهری آن است و بنابراین از حیث علمی قابل استناد نیست.><sep><اطلاعات ارائه شده بیشتر مربوط به قدمت ساخت قلعه بود.>
<برچسب>:
    تناقض

<این مزایا نتیجه کنگره یا ادارات و آژانس های فدرال است که توصیه های ما را برای کارآمدتر کردن خدمات دولت ، بهبود بودجه و هزینه های دلار مالیاتی و تقویت مدیریت منابع فدرال انجام می دهند.><sep><ما مسئول پیشرفت در مدیریت منابع فدرال هستیم.>
<برچسب>:
    تناظر

<دوستم اینگونه راضیم کرد که من از ارتفاعات فقط وقتی می‌ترسم که به آن فکر می‌کنم ولی وقتی آن بالا می‌رسم برطرف می‌شود.><sep><من از ارتفاعات می‌ترسیدم ولی دوستم مرا قانع کرد.>
<برچسب>:
    تناظر

<"یوتی‌ایر اوییشن"، یک شرکت هواپیمایی روسی است که در سال ۱۹۶۷ توسط خطوط هواپیمایی آئروفلوت تأسیس شد و در حال حاضر روزانه به ۷۲ مقصد، در آسیای مرکزی، آسیای جنوبی، آسیای جنوب شرقی، غرب آسیا و اروپا پروازهای مستقیم دارد.><sep><این شرکت یک شرکت دولتی است که بیش از پنجهزار نفر کارمند دارد. >
<برچسب>:
    ناشناخته


  فرمت خروجی:
  فرمت خروجی شما باید یک تاپل  باشد، که در آن هر تاپل از فرضیه و پیش فرض و پاسخ تشخیص داده شده تشکیل شده باشد.


برای نمونه تست زیر نام محتمل‌ترین برچسب را چاپ کن:

  """

**math and logic**

In [14]:
part = test_df[0:150]

In [21]:
part

,sent1,sent2,category,label
0,او را نیز بکشتند و پس از او، هیچ‌کس را نیافتند...,او را نیز بکشتند و پس از او پسرش را شایسته تری...,natural-wiki,c
1,آنها به عنوان (الف) رسید جبران خسارت طبقه بندی...,چیز خوبی است که آنها در این دسته قرار می گیرند.,translation-train,n
2,نعمت‌زاده همچنین با اشاره به ارتقاء کیفیت خودر...,به گفته‌ی وی تولید خودرو در مقایسه با سال‌های ...,natural-miras,n
3,صابر دین‌پژوه‌ رتبه اول علوم ریاضی و فنی کنکور...,رتبه‌ی اول علوم ریاضی و فنی کنکور از نتیجه‌ی ک...,natural-miras,c
4,مورد دیگری که باید ببینید ، فیلم خوابیدن با دش...,باید خوابیدن با دشمن را ببینی.,translation-train,e
...,...,...,...,...
145,برعکس ، افزایش پس انداز دولت باعث افزایش منابع...,برای افزایش پس انداز باید نرخ بهره افزایش یابد.,translation-dev,c
146,اکثر ED ها خدمات الکلی بسیار محدودی ارائه می د...,ED خدمات الكلی و بسیاری موارد دیگر را نیز ارائ...,translation-train,n
147,بجز فیل‌ها، گزارش‌هایی نیز وجود دارد مبنی بر ا...,فیلها موجودات عجیبی هستند. آنها حیواناتی بسیار...,natural-wiki,n
148,یا آیا آهن سرد حدس شما را در اینجا خراب می کند...,ساتر کارف در مورد آهن سرد و احتجاج سوالی ندارد.,translation-train,c


In [ ]:
import numpy as np

all_response = []

for index, row in part.iterrows():
    print('i: ',index)
    sent1 = row['sent1']
    sent2 = row['sent2']
    label = row['label']

    prompt_arman = f"""
    شرح وظیفه:
    {PROMPT_FEW}

     جمله اول:
    {sent1}

    <sep>

    جمله دوم:
    {sent2}

    پاسخ:
    """
    response = get_inference(prompt_arman)

    all_response.append({'sent1':sent1 ,'sent2':sent2, 'label':label , 'response':response})
    if index%10==0 :
        np.save(f'entailment_0_150_llama3_8b_arman_fewshot.npy', np.array(all_response, dtype=object))


i:  0
i:  1
